In [26]:

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import roc_auc_score, accuracy_score, f1_score, recall_score, precision_score, confusion_matrix
import pandas as pd
import numpy as np
import os
import time
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import StandardScaler
import os
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from tensorflow.keras.utils import to_categorical
from numpy import mean, std, dstack
from keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Conv1D, MaxPooling1D, LSTM, Dropout
from tensorflow.keras.utils import to_categorical
from sklearn import model_selection
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split
import time
import tensorflow as tf
import numpy as np
from keras.callbacks import EarlyStopping
from keras.layers import Dropout
from keras.regularizers import l2
from keras.optimizers import Adam
from keras.callbacks import LearningRateScheduler
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score, recall_score, precision_score
from sklearn.metrics import f1_score, recall_score, confusion_matrix,precision_score
from sklearn.utils import class_weight
from sklearn.metrics import precision_score

from sklearn.model_selection import StratifiedKFold
from tensorflow.keras.utils import to_categorical

from tensorflow.keras.layers import AveragePooling1D

from sklearn.neighbors import KNeighborsClassifier


def load_dataset(path):
    # Load input data and output data
    print('\nLoading dataset...')
    files = os.listdir(path)
    dataX_files = [file for file in files if 'x_' in file]
    datay_files = [file for file in files if 'y_' in file]
    dataX_files.sort()
    datay_files.sort()

    dataX = pd.concat([pd.read_csv(path + '/' + file, delimiter=',') for file in dataX_files])
    datay = pd.concat([pd.read_csv(path + '/' + file, delimiter=',') for file in datay_files])

    return dataX, datay

def feature_selection(path):
    dataX, datay = load_dataset(path)
    
    # Feature Engineering
    scaler = StandardScaler()
    dataX = scaler.fit_transform(dataX)
    
    # Feature extraction with logistic regression
    log_reg = LogisticRegression(max_iter=1000)
    log_reg.fit(dataX, datay)
    coefs = log_reg.coef_[0]
    
    # Pair the coefficients with their indices
    coef_index_pairs = list(zip(coefs, range(len(coefs))))
    
    # Sort the pairs by the absolute values of the coefficients
    sorted_pairs = sorted(coef_index_pairs, key=lambda pair: abs(pair[0]))
    
    selected_features = []
    roc_auc = 0  # Initialize roc_auc
    for coef, index in reversed(sorted_pairs):  # Iterate over the pairs from largest to smallest absolute coefficient
        selected_features.append(index)
        X_new = dataX[:, selected_features]
        X_train, X_test, y_train, y_test = train_test_split(X_new, datay, test_size=0.10, random_state=42)
        log_reg.fit(X_train, y_train)
        y_pred = log_reg.predict(X_test)
        new_roc_auc = roc_auc_score(y_test, y_pred)
        if new_roc_auc >= 0.9999:  # Stop if the ROC AUC score reaches 0.99
            break
        elif new_roc_auc <= roc_auc:  # Remove the feature if it doesn't improve the ROC AUC score
            selected_features.remove(index)
        else:
            roc_auc = new_roc_auc  # Update roc_auc if the ROC AUC score improves

    # Print the number of selected features
    print(f'Number of selected features: {len(selected_features)}')

    # Return the indices of the selected features, not the coefficients
    return selected_features



def load_3D_dataset(path, selected_features):
    # Load input data and output data
    print('\nLoading dataset...')
    files = os.listdir(path)
    dataX_files = [file for file in files if 'x_' in file]
    datay_files = [file for file in files if 'y_' in file]
    dataX_files.sort()
    datay_files.sort()

    dataX = pd.concat([pd.read_csv(path + '/' + file, delimiter=',') for file in dataX_files])
    
    # Select only the features in selected_features
    dataX = dataX.iloc[:, selected_features]

    datay = pd.DataFrame()  # Initialize datay as an empty DataFrame
    unique_label = 1
    for file in datay_files:
        datay_temp = pd.read_csv(path + '/' + file, delimiter=',')
        if (datay_temp.iloc[:, 0] == 1).any():  # Check if there are any 1 values in the first column
            datay_temp.loc[datay_temp.iloc[:, 0] == 1, datay_temp.columns[0]] = unique_label
            if datay.empty:  # If datay is empty, directly assign datay_temp
                datay = datay_temp
            else:
                datay = pd.concat([datay, datay_temp])  # Append only the modified datay_temp
            unique_label += 1

    # Split data into train and test sets
    trainX, testX = model_selection.train_test_split(dataX, test_size=0.10, random_state=42, shuffle=True)
    trainy, testy = model_selection.train_test_split(datay, test_size=0.10, random_state=42, shuffle=True)

    # Get 3D training data (assuming your data requires this step)
    listtrain = [trainX]
    listtest = [testX]
    trainX = dstack(listtrain)
    testX = dstack(listtest)
    print('\nAfter stacking...')
    print('trainX.shape: ', trainX.shape)
    print('testX.shape: ', testX.shape)

    # Calculate number of classes based on unique values in trainy
    num_classes = np.unique(trainy).shape[0]

    # Convert output data to categorical form
    trainy = to_categorical(trainy.values.ravel(), num_classes=num_classes)
    testy = to_categorical(testy.values.ravel(), num_classes=num_classes)
    print('\nAfter categorizing...')
    print('trainy.shape: ', trainy.shape)
    print('testy.shape: ', testy.shape, '\n')

    return trainX, trainy, testX, testy




from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier

def evaluate_model(model, trainX, trainy, testX, testy):
    # Flatten the data for the model
    trainX_flat = trainX.reshape(trainX.shape[0], -1)
    testX_flat = testX.reshape(testX.shape[0], -1)

    # Convert categorical labels back to 1-D for the model
    trainy_flat = np.argmax(trainy, axis=1)
    testy_flat = np.argmax(testy, axis=1)

    # Fit the model
    start_time = time.time()
    model.fit(trainX_flat, trainy_flat)
    training_time = time.time() - start_time

    # Predict the test set results
    start_time = time.time()
    y_pred = model.predict(testX_flat)
    testing_time = time.time() - start_time

    # Calculate Accuracy, F1 score, Recall, and Precision
    accuracy = accuracy_score(testy_flat, y_pred)
    f1 = f1_score(testy_flat, y_pred, average='weighted')
    recall = recall_score(testy_flat, y_pred, average='weighted')
    precision = precision_score(testy_flat, y_pred, average='weighted')

    # Compute confusion matrix
    cm = confusion_matrix(testy_flat, y_pred)

    print(f'Model: {model}')
    print(f'Accuracy : {accuracy}')
    print(f'F1 Score: {f1}')
    print(f'Recall: {recall}')
    print(f'Precision: {precision}')
    print(f'Confusion Matrix: \n{cm}\n')       
    print(f'Training time: {training_time} seconds')
    print(f'Testing time: {testing_time} seconds')

    return accuracy, f1, recall, precision, cm, training_time, testing_time

def run_experiment(datasetname, models):
    experiment_start_time = time.time()
    selected_features = feature_selection(datasetname)
    feature_extraction_time = time.time() - experiment_start_time
    trainX, trainy, testX, testy = load_3D_dataset(datasetname, selected_features)
    data_loading_time = time.time() - experiment_start_time - feature_extraction_time
    print('Evaluating for ' + datasetname + '...\n')
    scores = list()
    
    for model in models:
        print(f'Evaluating model: {model}')
        score = evaluate_model(model, trainX, trainy, testX, testy)
        scores.append(score)
    print("Duration of feature extraction: {} seconds".format(feature_extraction_time))
    print("Duration of loading data: {} seconds".format(data_loading_time))
    print("Duration of whole experiment: {} seconds".format(time.time() - experiment_start_time))

models = [KNeighborsClassifier(n_neighbors=3), RandomForestClassifier(max_depth=4), DecisionTreeClassifier(max_depth=4), AdaBoostClassifier()]
run_experiment('C:\\Users\\nalla\\OneDrive\\Provision_PT_838\\dataset\\Data', models)



Loading dataset...


C:\Users\nalla\anaconda3\Lib\site-packages\sklearn\utils\validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\nalla\anaconda3\Lib\site-packages\sklearn\utils\validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\nalla\anaconda3\Lib\site-packages\sklearn\utils\validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\nalla\anaconda3\Lib\site-packages\sklearn\utils\validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,

Number of selected features: 8

Loading dataset...

After stacking...
trainX.shape:  (1557162, 8, 1)
testX.shape:  (173018, 8, 1)

After categorizing...
trainy.shape:  (1557162, 11)
testy.shape:  (173018, 11) 

Evaluating for C:\Users\nalla\OneDrive\Provision_PT_838\dataset\Data...

Evaluating model: KNeighborsClassifier(n_neighbors=3)
Model: KNeighborsClassifier(n_neighbors=3)
Accuracy : 0.9898623264631425
F1 Score: 0.9898080025796862
Recall: 0.9898623264631425
Precision: 0.989807353580702
Confusion Matrix: 
[[98529     0     0     0     0     0     0     0     0     0     0]
 [    3  5305   333     0     0     0     0     0     0     2     0]
 [    1   433  2466     1     0     0     0     0     0     0     0]
 [    7     1     0  2808     0     0     0     0     0     0     3]
 [    8     0     0     0  8946     0     0     0     0     0     0]
 [    5     1     0     2     0 10400     0     0     0     0     0]
 [    0     0     0     0     0     0  5533     0     0   580    16]
 [

C:\Users\nalla\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Model: RandomForestClassifier(max_depth=4)
Accuracy : 0.9765978106324198
F1 Score: 0.9697216915091468
Recall: 0.9765978106324198
Precision: 0.9657896126195925
Confusion Matrix: 
[[98524     0     0     0     0     5     0     0     0     0     0]
 [    0  5643     0     0     0     0     0     0     0     0     0]
 [    2  2898     0     0     0     0     0     0     1     0     0]
 [   16     0     0  2800     0     0     3     0     0     0     0]
 [   12     0     0     0  8940     1     1     0     0     0     0]
 [   10     0     0     0     1 10397     0     0     0     0     0]
 [    1     0     0     0     0     0  5092     0   175   861     0]
 [    0     0     0     0     0     0     0  9626     7     0     0]
 [    0     0     0     0     0     0     0    53  6623     0     0]
 [    0     0     0     0     0     0     0     0     0 15748     0]
 [    1     0     0     0     0     0     0     0     0     1  5576]]

Training time: 149.16129970550537 seconds
Testing time: 1.506

C:\Users\nalla\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Model: DecisionTreeClassifier(max_depth=4)
Accuracy : 0.9141996786461524
F1 Score: 0.9017433068128714
Recall: 0.9141996786461524
Precision: 0.9108873418285587
Confusion Matrix: 
[[98529     0     0     0     0     0     0     0     0     0     0]
 [    0  5643     0     0     0     0     0     0     0     0     0]
 [    2  2899     0     0     0     0     0     0     0     0     0]
 [    2  2817     0     0     0     0     0     0     0     0     0]
 [    1     0     0     0  8952     1     0     0     0     0     0]
 [   10     0     0     0     1 10397     0     0     0     0     0]
 [    0  3514     0     0     0     0  2606     0     0     9     0]
 [    0     0     0     0     0     0     0  9633     0     0     0]
 [    0     0     0     0     0     0     2     0  6674     0     0]
 [    2     7     0     0     0     0     0     0     0 15739     0]
 [    1  5577     0     0     0     0     0     0     0     0     0]]

Training time: 5.2744081020355225 seconds
Testing time: 0.013

C:\Users\nalla\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [29]:

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import roc_auc_score, accuracy_score, f1_score, recall_score, precision_score, confusion_matrix
import pandas as pd
import numpy as np
import os
import time
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import StandardScaler
import os
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from tensorflow.keras.utils import to_categorical
from numpy import mean, std, dstack
from keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Conv1D, MaxPooling1D, LSTM, Dropout
from tensorflow.keras.utils import to_categorical
from sklearn import model_selection
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split
import time
import tensorflow as tf
import numpy as np
from keras.callbacks import EarlyStopping
from keras.layers import Dropout
from keras.regularizers import l2
from keras.optimizers import Adam
from keras.callbacks import LearningRateScheduler
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score, recall_score, precision_score
from sklearn.metrics import f1_score, recall_score, confusion_matrix,precision_score
from sklearn.utils import class_weight
from sklearn.metrics import precision_score

from sklearn.model_selection import StratifiedKFold
from tensorflow.keras.utils import to_categorical

from tensorflow.keras.layers import AveragePooling1D

from sklearn.neighbors import KNeighborsClassifier


def load_dataset(path):
    # Load input data and output data
    print('\nLoading dataset...')
    files = os.listdir(path)
    dataX_files = [file for file in files if 'x_' in file]
    datay_files = [file for file in files if 'y_' in file]
    dataX_files.sort()
    datay_files.sort()

    dataX = pd.concat([pd.read_csv(path + '/' + file, delimiter=',') for file in dataX_files])
    datay = pd.concat([pd.read_csv(path + '/' + file, delimiter=',') for file in datay_files])

    return dataX, datay

def feature_selection(path):
    dataX, datay = load_dataset(path)
    
    # Feature Engineering
    scaler = StandardScaler()
    dataX = scaler.fit_transform(dataX)
    
    # Feature extraction with logistic regression
    log_reg = LogisticRegression(max_iter=1000)
    log_reg.fit(dataX, datay)
    coefs = log_reg.coef_[0]
    
    # Pair the coefficients with their indices
    coef_index_pairs = list(zip(coefs, range(len(coefs))))
    
    # Sort the pairs by the absolute values of the coefficients
    sorted_pairs = sorted(coef_index_pairs, key=lambda pair: abs(pair[0]))
    
    selected_features = []
    roc_auc = 0  # Initialize roc_auc
    for coef, index in reversed(sorted_pairs):  # Iterate over the pairs from largest to smallest absolute coefficient
        selected_features.append(index)
        X_new = dataX[:, selected_features]
        X_train, X_test, y_train, y_test = train_test_split(X_new, datay, test_size=0.10, random_state=42)
        log_reg.fit(X_train, y_train)
        y_pred = log_reg.predict(X_test)
        new_roc_auc = roc_auc_score(y_test, y_pred)
        if new_roc_auc >= 0.9999:  # Stop if the ROC AUC score reaches 0.99
            break
        elif new_roc_auc <= roc_auc:  # Remove the feature if it doesn't improve the ROC AUC score
            selected_features.remove(index)
        else:
            roc_auc = new_roc_auc  # Update roc_auc if the ROC AUC score improves

    # Print the number of selected features
    print(f'Number of selected features: {len(selected_features)}')

    # Return the indices of the selected features, not the coefficients
    return selected_features



def load_3D_dataset(path, selected_features):
    # Load input data and output data
    print('\nLoading dataset...')
    files = os.listdir(path)
    dataX_files = [file for file in files if 'x_' in file]
    datay_files = [file for file in files if 'y_' in file]
    dataX_files.sort()
    datay_files.sort()

    dataX = pd.concat([pd.read_csv(path + '/' + file, delimiter=',') for file in dataX_files])
    
    # Select only the features in selected_features
    dataX = dataX.iloc[:, selected_features]

    datay = pd.DataFrame()  # Initialize datay as an empty DataFrame
    unique_label = 1
    for file in datay_files:
        datay_temp = pd.read_csv(path + '/' + file, delimiter=',')
        if (datay_temp.iloc[:, 0] == 1).any():  # Check if there are any 1 values in the first column
            datay_temp.loc[datay_temp.iloc[:, 0] == 1, datay_temp.columns[0]] = unique_label
            if datay.empty:  # If datay is empty, directly assign datay_temp
                datay = datay_temp
            else:
                datay = pd.concat([datay, datay_temp])  # Append only the modified datay_temp
            unique_label += 1

    # Split data into train and test sets
    trainX, testX = model_selection.train_test_split(dataX, test_size=0.10, random_state=42, shuffle=True)
    trainy, testy = model_selection.train_test_split(datay, test_size=0.10, random_state=42, shuffle=True)

    # Get 3D training data (assuming your data requires this step)
    listtrain = [trainX]
    listtest = [testX]
    trainX = dstack(listtrain)
    testX = dstack(listtest)
    print('\nAfter stacking...')
    print('trainX.shape: ', trainX.shape)
    print('testX.shape: ', testX.shape)

    # Calculate number of classes based on unique values in trainy
    num_classes = np.unique(trainy).shape[0]

    # Convert output data to categorical form
    trainy = to_categorical(trainy.values.ravel(), num_classes=num_classes)
    testy = to_categorical(testy.values.ravel(), num_classes=num_classes)
    print('\nAfter categorizing...')
    print('trainy.shape: ', trainy.shape)
    print('testy.shape: ', testy.shape, '\n')

    return trainX, trainy, testX, testy




from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier

def evaluate_model(model, trainX, trainy, testX, testy):
    # Flatten the data for the model
    trainX_flat = trainX.reshape(trainX.shape[0], -1)
    testX_flat = testX.reshape(testX.shape[0], -1)

    # Convert categorical labels back to 1-D for the model
    trainy_flat = np.argmax(trainy, axis=1)
    testy_flat = np.argmax(testy, axis=1)

    # Fit the model
    start_time = time.time()
    model.fit(trainX_flat, trainy_flat)
    training_time = time.time() - start_time

    # Predict the test set results
    start_time = time.time()
    y_pred = model.predict(testX_flat)
    testing_time = time.time() - start_time

    # Calculate Accuracy, F1 score, Recall, and Precision
    accuracy = accuracy_score(testy_flat, y_pred)
    f1 = f1_score(testy_flat, y_pred, average='weighted')
    recall = recall_score(testy_flat, y_pred, average='weighted')
    precision = precision_score(testy_flat, y_pred, average='weighted')

    # Compute confusion matrix
    cm = confusion_matrix(testy_flat, y_pred)

    print(f'Model: {model}')
    print(f'Accuracy : {accuracy}')
    print(f'F1 Score: {f1}')
    print(f'Recall: {recall}')
    print(f'Precision: {precision}')
    print(f'Confusion Matrix: \n{cm}\n')       
    print(f'Training time: {training_time} seconds')
    print(f'Testing time: {testing_time} seconds')

    return accuracy, f1, recall, precision, cm, training_time, testing_time

def run_experiment(datasetname, models):
    experiment_start_time = time.time()
    selected_features = feature_selection(datasetname)
    feature_extraction_time = time.time() - experiment_start_time
    trainX, trainy, testX, testy = load_3D_dataset(datasetname, selected_features)
    data_loading_time = time.time() - experiment_start_time - feature_extraction_time
    print('Evaluating for ' + datasetname + '...\n')
    scores = list()
    
    for model in models:
        print(f'Evaluating model: {model}')
        score = evaluate_model(model, trainX, trainy, testX, testy)
        scores.append(score)
    print("Duration of feature extraction: {} seconds".format(feature_extraction_time))
    print("Duration of loading data: {} seconds".format(data_loading_time))
    print("Duration of whole experiment: {} seconds".format(time.time() - experiment_start_time))

models = [KNeighborsClassifier(n_neighbors=2),KNeighborsClassifier(n_neighbors=250)]
run_experiment('C:\\Users\\nalla\\OneDrive\\Provision_PT_838\\dataset\\Data', models)



Loading dataset...


C:\Users\nalla\anaconda3\Lib\site-packages\sklearn\utils\validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\nalla\anaconda3\Lib\site-packages\sklearn\utils\validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\nalla\anaconda3\Lib\site-packages\sklearn\utils\validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\nalla\anaconda3\Lib\site-packages\sklearn\utils\validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,

Number of selected features: 8

Loading dataset...

After stacking...
trainX.shape:  (1557162, 8, 1)
testX.shape:  (173018, 8, 1)

After categorizing...
trainy.shape:  (1557162, 11)
testy.shape:  (173018, 11) 

Evaluating for C:\Users\nalla\OneDrive\Provision_PT_838\dataset\Data...

Evaluating model: KNeighborsClassifier(n_neighbors=2)
Model: KNeighborsClassifier(n_neighbors=2)
Accuracy : 0.9899663618814227
F1 Score: 0.9899018936164452
Recall: 0.9899663618814227
Precision: 0.9901751472870328
Confusion Matrix: 
[[98529     0     0     0     0     0     0     0     0     0     0]
 [    2  5494   145     0     0     1     0     0     0     1     0]
 [    0   601  2298     2     0     0     0     0     0     0     0]
 [    6     1     0  2809     0     1     0     0     0     0     2]
 [    8     0     0     0  8946     0     0     0     0     0     0]
 [    5     2     0     1     0 10400     0     0     0     0     0]
 [    0     0     0     0     0     0  5779     0     0   345     5]
 